<a href="https://colab.research.google.com/github/MapleWolfe/Anomaly_detection_waymo_open/blob/main/anomoly_autoencoder_model/autoencoder_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Model for Waymo open dataset
- please see the auto_encoder_readme.md

##### Enviornment Notebook was built in:
- Kindly run this notebook in Google Colab

##### Hardware needs:

- Kindly use a minimum of 12 GB CPU ram
- the standard Nvidia T4 GPU that comes with the free version of Google Colab.
- 50 GB disk Minimum (less should work but this was the minimum during notebook creation)

##### following files should be in local directory if using the local notebook
- these files should be in the same folder as notebook

##### instructions to run notebook,
- Kindly fill in the code cell below to use the local parquet files stored in the same folder as notebook or the parquet files stored in a GCS bucket

In [33]:
# do you want to use local sample files or google cloud storage?
file_store = 'GCS' # kindly use either 'GCS' for Google cloud storage or 'LOCAL' for files stored in the same folder as notebook

# leave both as None if using LOCAL or kindly replace this with your GCS api key json file and Bucket name or else
gcs_json_key = '/content/organic-reef-390716-609989a4c6da.json' # please remember to start file path here with '/content/' + your json key file name
bucket_name = 'waymo_sample_bucket'

#these are the file download limits, it limits the number of parquet files being downloaded for training / validation / test
train_limit = 2 #-1 means all relevant files in your google bucket
val_limit = 1 #-1 means all relevant files in your google bucket
test_limit = 1 #-1 means all relevant files in your google bucket


# the path below is to the sample files: there are 4 parquets in total, you just need to upload them into colab enviornment
train_box_data_file_path  = 'training_camera_box_10017090168044687777_6380_000_6400_000.parquet'
train_image_data_file_path = 'training_camera_image_10017090168044687777_6380_000_6400_000.parquet'
val_box_data_file_path = 'training_camera_box_10017090168044687777_6380_000_6400_000.parquet'
val_image_data_file_path = 'training_camera_image_10017090168044687777_6380_000_6400_000.parquet'


## installs, imports, pre-sets

- kindly open and run cell blocks based on the enviornment being run in to save computational resources.

### Using detect_model_requirements.txt
- please uncomment to use

In [2]:
# uncomment to create notebook package requirments file called detect_model_requirements.txt
#!pip freeze > detect_model_requirements.txt

# use the code below to use detect_model_requirements.txt to install all necessary packages
#!pip install -r detect_model_requirements.txt

### Neccessary installs on top of google colab
- uncomment and Run this cell if you aren't using detect_model_requirements.txt and are operating in the GPU google colab enviornment

In [52]:
#!pip install google-cloud-storage
#!pip install altair

1

### Imports

In [3]:
# installs for google cloud storage
from google.cloud import storage

# general tool installs
import os, io, shutil, warnings
from tqdm.notebook import tqdm

#image processing and plotting libraries
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# data processing librarires
import pandas as pd
import numpy as np

# model evaluation
from sklearn.model_selection import ParameterGrid

# model libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#pre-sets
warnings.filterwarnings("ignore")
physical_devices = tf.config.list_physical_devices('GPU')
print('GPU_count:' ,len(physical_devices))

GPU_count: 1


## utility functions
- functions that may be used at a later point

### delete file function

In [4]:
# code to delete a file
def delete_file(file_path_list):
  for a_file in (file_path_list):
    os.remove(a_file)
  return None

### move copy function

In [5]:
def move_copy(old_location, new_location):
  shutil.copy(old_location, new_location)
  return None

### download blob function

In [6]:
# to download a blob file
def download_blob(a_blob,file_name):
  a_blob.download_to_filename(file_name)
  return None

### Let's create folder directory for Autoencoder

In [7]:
#folder name list for directory
def make_directory(folder_name_list):
  folder_path = os.path.join(*folder_name_list)
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  return None

#function to build directory structure
def build_a_directory():
  for folder_type in ['train','test','eval']:
    for data_type in ['images']:
      make_directory(['datasets',folder_type,data_type])
  return None



In [8]:
# running directory function
build_a_directory()

## Google Cloud Storage section

- Run these cells if you are using your private google cloud storage.
- kindly ensure that your bucket has the same structure and file names as the waymo_open_dataset_v_2_0_0 bucket

In [9]:
def start_gcs(gcs_json_key = gcs_json_key, bucket_name = bucket_name):
# Please input API JSON KEY FOR your private google cloud storage where the files are kept in gcs_json_key defined in the first codeblock
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = gcs_json_key
    client = storage.Client()
# replace the bucket name with the bucket name in your GCS, use the bucket_name object  in the first codeblock
    bucket = client.get_bucket(bucket_name)

#getting file list
    files = bucket.list_blobs()
    files_list = [a_file.name for a_file in files]
    return bucket, files_list


### GCS Iterative File Dowload function
(Bare Image parquet and Bounding box parquet files)

In [10]:
# in the following function we are creating blobs: one blob object for images parquet and another blob for box coordinates parquet
def open_gcs_file(files_list,bucket, storage_folder = 'training'):
    # lets ensure that we have the correct file type
    if storage_folder not in ['training','testing','validation']:
      print('''please retype storage_folder it should either be ('training','testing','validation') in the second parameter of function)''')
      return None

    image_box_str = storage_folder +'/'+ 'camera_box'+'/'
    bare_image_str = storage_folder +'/'+ 'camera_image'+'/'
    print('files_list: ', files_list)
    for a_file in files_list:
        if image_box_str in a_file:
          try:
            box_file_name = a_file
            box_file_blob = bucket.blob(box_file_name)
            bare_image_file_name = box_file_name.replace(image_box_str, bare_image_str)
            bare_image_blob = bucket.blob(bare_image_file_name)
            if bare_image_blob is not None:
              yield box_file_blob, os.path.basename(box_file_name),bare_image_blob, os.path.basename(bare_image_file_name)
          except:
            continue


## Storing files for autoencoder model
- functions to unpack and store from parquet files

### save path and create unique key function

In [11]:
# creating a save path
def create_path(file_id,store_type, images_labels = 'images'):
  if store_type == 'train':
    if images_labels == 'images':
      return 'datasets/train/images/' +str(file_id) + '.jpg'
    else:
      return 'datasets/train/labels/' +str(file_id) + '.txt'

  elif store_type == 'eval':
    if images_labels == 'images':
      return 'datasets/eval/images/' +str(file_id) + '.jpg'
    else:
      return 'datasets/eval/labels/' +str(file_id) + '.txt'
  elif store_type == 'train':
    if images_labels == 'images':
      return 'datasets/train/images/' +str(file_id) + '.jpg'
    else:
      return 'datasets/train/labels/' +str(file_id) + '.txt'
  else:
    print('error type parameter: train / eval / test')

# creating a key column
def create_key_column(select_df):
  return select_df['key.segment_context_name'].astype(str) +  select_df['key.frame_timestamp_micros'].astype(str) + select_df['key.camera_name'].astype(str)


### Building image pre-processing functions

In [12]:
# this effectively converts all pixels within the bounding box into black pixels
def blackout_bounding_boxes(df, image_array):
    for index, row in df.iterrows():
        center_x = int(row['center_x'])
        center_y = int(row['center_y'])
        width = int(row['size_x'])
        height = int(row['size_y'])

        x1 = int(center_x - width / 2)
        y1 = int(center_y - height / 2)
        x2 = int(center_x + width / 2)
        y2 = int(center_y + height / 2)

        image_array[y1:y2, x1:x2] = 0

    return image_array

# this will break the image into smaller images to be stacked

def split_and_stack_image(image_array, output_shape=(64, 64)):
    input_height, input_width = image_array.shape
    num_rows = input_height // output_shape[0]
    num_cols = input_width // output_shape[1]
    stacked_images = []
    for i in range(num_rows):
        for j in range(num_cols):
            start_row = i * output_shape[0]
            end_row = start_row + output_shape[0]
            start_col = j * output_shape[1]
            end_col = start_col + output_shape[1]

            small_image = image_array[start_row:end_row, start_col:end_col]
            stacked_images.append(small_image)

    stacked_array = np.stack(stacked_images, axis = 0)
    return stacked_array


#### Unpacking, scaling, storing images and annotations seperately

In [13]:
# building bounding box labels
# here we are also scaling the bounding box positions to the new image of 640 x 640
def build_save_labels(df,image_size):
  width, height = image_size
  df.loc[:,'center_x'] = (df.loc[:,'[CameraBoxComponent].box.center.x'] * (640/width))
  df.loc[:,'center_y'] = (df.loc[:,'[CameraBoxComponent].box.center.y'] * (640/height))
  df.loc[:,'size_x'] = (df.loc[:,'[CameraBoxComponent].box.size.x'] * (640/width))
  df.loc[:,'size_y'] = (df.loc[:,'[CameraBoxComponent].box.size.y'] * (640/height))

  select_col_df = df[['[CameraBoxComponent].type','center_x','center_y','size_x','size_y']]
  return select_col_df



# constructing and saving image to path we are
# here we are checking the image size and resizing it to 640 x 640
# here we are also converting images to grayscale
# here weare also
def build_save_image(byte_string, box_df):
  image_bytes = io.BytesIO(byte_string)
  image = Image.open(image_bytes)
  original_image_size = image.size

  if (original_image_size[0] %32 == 0) and (original_image_size[1] %32 == 0):
    #creating scaled bounding boxes
    bounding_box_df = build_save_labels(box_df,original_image_size)

    # resizing and greyscaling
    resized_image = image.resize((640,640))
    gray_img = np.mean(np.array(resized_image), axis=2, dtype=np.uint8)
    #blackout areas within bounding boxes
    blackout_img_array = blackout_bounding_boxes(bounding_box_df, gray_img)
    split_img_array = split_and_stack_image(blackout_img_array, output_shape=(64, 64))
    return split_img_array
  else:
    return 'fail'




In [28]:
# combining all functions above
def unpack_store_images(image_parquet,box_parquet, store_type = 'train'):
  file_path_store = []
  image_df = pd.read_parquet(image_parquet)
  box_df = pd.read_parquet(box_parquet)

  image_df.loc[:,'key_column'] = create_key_column(image_df)
  box_df.loc[:,'key_column'] = create_key_column(box_df)

  commmon_key_list = list(set(box_df['key_column'].tolist()) & set(image_df['key_column'].tolist()))
  id_iterator = 0
  for common_key in tqdm(commmon_key_list):
    key_loc_image = image_df.loc[image_df['key_column'] == common_key]
    image_bytes = key_loc_image.reset_index().loc[0, '[CameraImageComponent].image']
    key_loc_box = box_df.loc[box_df['key_column'] == common_key]

    image_iterator = build_save_image(image_bytes,key_loc_box)
    if image_iterator != 'fail':
      for image_array in image_iterator:
        image_path = create_path(id_iterator,store_type, images_labels = 'images')
        id_iterator +=1
        file_path_store.append(image_path)
        output_image = Image.fromarray(image_array)
        output_image.save(image_path)
  return file_path_store

## building model

In [32]:
class AutoEncoder(tf.keras.Model):
    def __init__(self, code_size=4, input_shape=(64, 64,1)):
        super().__init__()
        decoder_output_shape = input_shape[0] * input_shape[1]*1

        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(code_size, activation='relu')
        ])

        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(decoder_output_shape, activation='sigmoid')
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded


In [57]:
def train_model(file_counter, model_path):
  batch_size = 20
  data_generator = ImageDataGenerator(rescale=1.0/255)
  train_data = data_generator.flow_from_directory('/content/datasets/train/',batch_size= batch_size,class_mode='input', color_mode='grayscale', target_size=(64, 64))
  validation_data = data_generator.flow_from_directory('/content/datasets/eval/',batch_size= batch_size,class_mode='input', color_mode='grayscale', target_size=(64, 64))
  if file_counter == 0:
    model = AutoEncoder(code_size=64)
    model.compile(loss='msle', metrics=['mse'], optimizer='adam')

    fitted_encoder_history = model.fit(train_data, steps_per_epoch=train_data.samples // batch_size,
      epochs=1,validation_data=validation_data, validation_steps=validation_data.samples // batch_size)

    model.save(model_path,save_format="tf" )
    return model_path

  else:
    model = tf.keras.models.load_model(model_path)
    model.compile(loss='msle', metrics=['mse'], optimizer='adam')

    fitted_encoder_history = model.fit(train_data, steps_per_epoch=train_data.samples // batch_size,
      epochs=3,validation_data=validation_data, validation_steps=validation_data.samples // batch_size)


    return model, model_path


## Creating validation data

In [ ]:
# checking and then initiating GCS file download
if file_store == 'GCS':
  bucket, files_list = start_gcs()
  gcs_iterator = open_gcs_file(files_list,bucket, storage_folder = 'validation')
  file_counter = 0
  train_file_path_list = []
  for a_file in gcs_iterator:
    # evaluation data
    box_file_blob, box_file_name,bare_image_blob, bare_image_file_name = a_file

    download_blob(box_file_blob,box_file_name)
    new_box_file_name = 'box_file_'+ box_file_name
    os.rename(box_file_name, new_box_file_name)

    download_blob(bare_image_blob,bare_image_file_name)
    new_image_file_name = 'image_file_'+ bare_image_file_name
    os.rename(bare_image_file_name, new_image_file_name)

    eval_file_path_list = unpack_store_images(new_image_file_name,new_box_file_name, store_type = 'eval')
    delete_file([new_box_file_name,new_image_file_name])

    print('completed_file_number: ',file_counter)
    file_counter += 1
    if (file_counter >= val_limit) and (val_limit != -1):
      break

elif file_store == 'LOCAL':
    train_file_path_list = unpack_store_images(val_box_data_file_path,val_image_data_file_path, store_type = 'eval')
else:
  print('''check file_store variable, it should be in capital 'GCS' or 'LOCAL' ''')


## main run to create data store and train model

In [ ]:
# checking and then initiating GCS file download
model_path = 'encoder_model'
if file_store == 'GCS':
  bucket, files_list = start_gcs()
  gcs_iterator = open_gcs_file(files_list,bucket, storage_folder = 'training')
  file_counter = 0
  train_file_path_list = []
  for a_file in gcs_iterator:
    # train data
      box_file_blob, box_file_name,bare_image_blob, bare_image_file_name = a_file

      download_blob(box_file_blob,box_file_name)
      new_box_file_name = 'box_file_'+ box_file_name
      os.rename(box_file_name, new_box_file_name)

      download_blob(bare_image_blob,bare_image_file_name)
      new_image_file_name = 'image_file_'+ bare_image_file_name
      os.rename(bare_image_file_name, new_image_file_name)

      train_file_path_list = unpack_store_images(new_image_file_name,new_box_file_name, store_type = 'train')

      model_path = train_model(file_counter, model_path)
      print('completed_file_number: ',file_counter)
      file_counter += 1

      delete_file([new_box_file_name,new_image_file_name]+train_file_path_list)
      if (file_counter >= train_limit) and (train_limit != -1):
        break

elif file_store == 'LOCAL':
    file_counter = 0
    model_list = []
    train_file_path_list = unpack_store_images(train_box_data_file_path,train_image_data_file_path, store_type = 'train')

    model_list = train_model(file_counter, model_path)
    delete_file([train_box_data_file_path,train_image_data_file_path]+train_file_path_list)
else:
  print('''check file_store variable, it should be in capital 'GCS' or 'LOCAL' ''')


## Model evaluation and comparison

## Bounding Box Building Functions

In [ ]:
#to plot one box
def plot_a_box(box_spec_list):
  print(box_spec_list)
  center_x = box_spec_list[0]
  center_y = box_spec_list[1]
  width = box_spec_list[2]
  height =box_spec_list[3]
  x_min = center_x - (width / 2)
  y_min = center_y - (height / 2)
  rect = patches.Rectangle((x_min, y_min), width, height,linewidth=2, edgecolor='gold', facecolor='none')
  return rect

# to plot all boxes
def plot_boxes(image_object,box_iterator_object):
  fig, ax = plt.subplots()
  ax.imshow(image_object)
  for a_box in box_iterator_object:
    one_box_val = a_box.xywh[0].tolist()
    one_plot_box = plot_a_box(one_box_val)
    ax.add_patch(one_plot_box)
  ax.set_xlim(0, image_object.width)
  return fig, ax

## Model building

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
a_image = one_image_df.loc[0,'[CameraImageComponent].image']
image_bytes = io.BytesIO(a_image)
image = Image.open(image_bytes)
print(type(image))

<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [ ]:
results = model(image)
for result in results:
  boxes = result.boxes  # Boxes object for bbox outputs
  fig, ax = plot_boxes(image,boxes)
  plt.show()

In [ ]:
boxes[0].xywh[0].tolist()

[1090.59765625, 543.5984497070312, 149.1917724609375, 142.18844604492188]